In [2]:
from google.cloud import bigquery
import pandas as pd

# initiating client for bigquery and setting location
client = bigquery.Client(location="europe-west2")

In [40]:
query = ("""
SELECT

UPPER(guid) AS guid
UPPER(address) AS address
UPPER(current_energy_rating) AS current_energy_rating
UPPER(potential_energy_rating) AS potential_energy_rating
UPPER(construction_age_band) AS construction_age_band

FROM ons-ids-analysis-prod.demo_wip_notebook.epc_filtered1
""")

query_job = client.query(
    query,
    # Location must match that of the dataset(s) referenced in the query.
    location="europe-west2",
)  # API request - starts the query

In [41]:
df = query_job.to_dataframe()

In [31]:
 # Construct a reference to the wip notebook dataset, within the ons-ids-analysis-prod project
    
PROJECT_ID = 'ons-ids-analysis-prod'
dataset_ref = client.dataset('demo_wip_notebook', project=PROJECT_ID)

In [32]:
# Make an API request to fetch the dataset
dataset = client.get_dataset(dataset_ref)

In [34]:
sql = """
SELECT *
WHERE

FROM ons-ids-analysis-prod.demo_wip_notebook.epc_filtered1
"""

table_ref = dataset.table("epc_upper")
job_config = bigquery.QueryJobConfig(
    destination=table_ref
)

# Start the query, passing in the extra configuration.
query_job = client.query(sql, location="europe-west2", job_config=job_config)

query_job.result()  # Waits for the query to finish
print("Query results loaded to table {}".format(table_ref.path))

Query results loaded to table /projects/ons-ids-analysis-prod/datasets/demo_wip_notebook/tables/epc_upper


In [10]:
# set data types
# ensure energy rating e.g. is a capital letter between A-E
# upper case variables
# strip leading/trailing spaces
# strip non-alphanumeric characters

In [42]:
df

,guid
0,7B4EF54B-BF70-408E-9DC3-F038988FDD5B
1,535B2592-1AB0-4518-9B12-15C4DF55D5D9
2,E06DE197-D667-4BA0-A5A1-9D471CB8318F
3,EF9B99CA-524A-41D1-B853-BD587EE9EC86
4,B09CEC47-7C96-4447-B6DE-E443594CEBD3
...,...
850361,B1C41541-7E7D-4D62-998D-00E49084D3CD
850362,0AE5E4C2-190B-467D-91B2-780420B76F8C
850363,FC4C8C2B-D344-490A-9414-A4E5E0CF62E9
850364,2B74A722-1112-4DCF-A7C4-D6A50BB1DAB2


In [5]:
# we can then pass this to a pandas dataframe
df = query_job.to_dataframe()
'''
However, when we get to actually use the IDS properly, we should do this sparingly.

BigQuery does lots of magic, efficiency-saving stuff in the background that makes doing
your data processing in BigQuery, where possible, a much better proposition than doing so
on a spark or pandas dataframe.

This probably means we're going to have to get really good at SQL.
'''

"\nHowever, when we get to actually use the IDS properly, we should do this sparingly.\n\nBigQuery does lots of magic, efficiency-saving stuff in the background that makes doing\nyour data processing in BigQuery, where possible, a much better proposition than doing so\non a spark or pandas dataframe.\n\nThis probably means we're going to have to get really good at SQL.\n"

In [9]:
df['posttown'].value_counts()

BRISTOL                 12553
MANCHESTER              11829
LEICESTER               11492
NOTTINGHAM              11245
LIVERPOOL               10723
                        ...  
RICKINGHALL INFERIOR        1
CWMGORS                     1
KEWSTOKE                    1
PERTENHALL                  1
Brixham                     1
Name: posttown, Length: 5576, dtype: int64

In [ ]:
# create tiny geography index
GI = pd.DataFrame({'local_authority': [],
                   'geography_entry_id': []
                  })



In [ ]:
# join to geography index
df = pd.merge(GI, on = 'geography_entry_id', how = 'inner')

In [ ]:
# analysis

#